# Semantic Gravity Experiment Pipeline

This notebook runs the complete Semantic Gravity experiment pipeline:
1. Setup and dependencies
2. Inference (greedy + sampling)
3. Behavior analysis
4. Mechanistic metrics
5. Activation patching
6. Bootstrap CIs
7. Visualization

In [ ]:
# Install dependencies
!pip install -q torch transformers accelerate tokenizers numpy pandas scipy scikit-learn matplotlib tqdm requests wordfreq datasets


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# =============================================================================
# CONFIGURATION - UPDATE THESE PATHS FOR YOUR SETUP
# =============================================================================
# REPO_DIR: Path to the cloned Semantic Gravity repository in Google Drive
# MODEL_DIR: Path to model weights (e.g., Qwen2.5-7B-Instruct)
#            If using HuggingFace model ID, set to the model ID string
# =============================================================================

REPO_DIR = '/content/drive/MyDrive/Semantic_Gravity'  # Path to repo in Drive

# Model path: either local path in Drive or HuggingFace model ID
# Default from CONFIG: 'Qwen/Qwen2.5-7B-Instruct' (will download from HF)
MODEL_DIR = None  # Set to None to use CONFIG default, or specify path/model ID

# Validate repository directory exists
if not os.path.exists(REPO_DIR):
    raise FileNotFoundError(
        f'Repository directory not found: {REPO_DIR}\n'
        f'Please update REPO_DIR to point to your cloned Semantic Gravity repository.'
    )

os.chdir(REPO_DIR)

# Load CONFIG to get default model path if not specified
import sys
sys.path.insert(0, REPO_DIR)
from src.config import CONFIG, get_base_paths

if MODEL_DIR is None:
    MODEL_DIR = CONFIG.get('model', {}).get('model_id', 'Qwen/Qwen2.5-7B-Instruct')
    print(f'Using model from CONFIG: {MODEL_DIR}')

# Validate model path if it's a local directory
if os.path.sep in MODEL_DIR and not os.path.exists(MODEL_DIR):
    raise FileNotFoundError(
        f'Model directory not found: {MODEL_DIR}\n'
        f'If using a local model, ensure the path exists.\n'
        f'If using a HuggingFace model ID, set MODEL_DIR to just the model ID '
        f'(e.g., "Qwen/Qwen2.5-7B-Instruct").'
    )

os.environ['SEMANTIC_GRAVITY_MODEL_PATH'] = MODEL_DIR

print(f'Working directory: {os.getcwd()}')
print(f'Model: {MODEL_DIR}')

In [ ]:
# Check GPU (A100 required)
!nvidia-smi -L

import torch
if not torch.cuda.is_available():
    raise RuntimeError('No GPU available')
gpu_name = torch.cuda.get_device_name(0)
print(f'GPU: {gpu_name}')
if 'A100' not in gpu_name:
    raise RuntimeError(f'A100 required, found: {gpu_name}')


In [ ]:
import sys

sys.path.insert(0, REPO_DIR)

from src.utils import set_all_seeds, ModelWrapper
from src.config import validate_environment, CONFIG

set_all_seeds()
print('Repository added to path and seeds set')


In [ ]:
# OPTIONAL: Run dataset pipeline (uncomment if needed)
# from src.dataset_pipeline import build_dataset
# build_dataset()

In [ ]:
# Run inference
import json
import os
import subprocess

from src.config import setup_directories, validate_environment, CONFIG
from src.utils import ModelWrapper

RUN_ROOT = setup_directories()['run_root']
print(f'Run root: {RUN_ROOT}')

metadata = validate_environment()
metadata['model_path'] = os.environ.get('SEMANTIC_GRAVITY_MODEL_PATH', '')
metadata['model_id'] = CONFIG.get('model', {}).get('model_id', '')

try:
    git_commit = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=REPO_DIR).decode().strip()
except Exception:
    git_commit = 'unknown'
metadata['git_commit'] = git_commit

wrapper = ModelWrapper.get_instance()
if not wrapper.is_loaded:
    wrapper.load()
metadata['tokenizer_vocab_size'] = len(wrapper.tokenizer)

metadata_path = os.path.join(RUN_ROOT, 'run_metadata.json')
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2)
print(f'Saved run metadata to {metadata_path}')

from src.runner import run_experiment
run_experiment(output_root=RUN_ROOT, limit=None)


In [ ]:
# Run behavior analysis
from src.behavior_analysis import run_behavior_analysis_pipeline
run_behavior_analysis_pipeline(output_root=RUN_ROOT)

In [ ]:
# Run mechanistic metrics
from src.metrics_attn import run_mechanistic_metrics_pipeline
run_mechanistic_metrics_pipeline(output_root=RUN_ROOT)

In [ ]:
# Run activation patching
from src.patching import run_activation_patching_pipeline
run_activation_patching_pipeline(output_root=RUN_ROOT)

In [ ]:
# Run bootstrap CIs and visualization
from src.bootstrap import run_bootstrap_pipeline
from src.visualize import run_visualization_pipeline

run_bootstrap_pipeline(output_root=RUN_ROOT)
result_paths = run_visualization_pipeline(output_root=RUN_ROOT)

print('\nGenerated outputs:')
for name, path in result_paths.items():
    print(f'  {name}: {path}')